# Exploratory Data Analysis (EDA)
## Agriculture Crop Production Dataset

This notebook performs comprehensive exploratory data analysis on the agricultural crop production dataset from data.gov.in.

**Dataset Period:** 2001-2014  
**Source:** data.gov.in (Government of India)


In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("Libraries imported successfully!")


## 1. Data Loading


In [ ]:
# Load data
data_path = Path("../data/raw/crop_production_data.csv")

# Try to load the dataset
try:
    df = pd.read_csv(data_path)
    print(f"✅ Dataset loaded successfully!")
    print(f"Shape: {df.shape}")
except FileNotFoundError:
    print("⚠️ Dataset file not found. Using sample data generation...")
    # Generate sample data for demonstration
    import sys
    sys.path.append('../src')
    from utils.data_loader import load_data
    df = load_data()
    print(f"✅ Sample data generated!")
    print(f"Shape: {df.shape}")

# Display first few rows
df.head()


## 2. Dataset Overview


In [ ]:
# Basic information
print("=" * 60)
print("DATASET OVERVIEW")
print("=" * 60)
print(f"\nTotal Records: {len(df):,}")
print(f"Total Features: {len(df.columns)}")
print(f"\nColumn Names:")
for i, col in enumerate(df.columns, 1):
    print(f"  {i}. {col}")

print(f"\nData Types:")
print(df.dtypes)

print(f"\nMemory Usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")


## 3. Missing Values Analysis


In [ ]:
# Check for missing values
missing_data = df.isnull().sum()
missing_percent = (missing_data / len(df)) * 100

missing_df = pd.DataFrame({
    'Column': missing_data.index,
    'Missing Count': missing_data.values,
    'Missing Percentage': missing_percent.values
})
missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values('Missing Count', ascending=False)

if len(missing_df) > 0:
    print("Missing Values Found:")
    print(missing_df.to_string(index=False))
    
    # Visualize missing values
    plt.figure(figsize=(10, 6))
    sns.barplot(data=missing_df, x='Missing Percentage', y='Column')
    plt.title('Missing Values by Column')
    plt.xlabel('Missing Percentage (%)')
    plt.tight_layout()
    plt.show()
else:
    print("✅ No missing values found in the dataset!")


## 4. Statistical Summary


In [ ]:
# Statistical summary for numerical columns
numerical_cols = df.select_dtypes(include=[np.number]).columns
print("Statistical Summary for Numerical Columns:")
print("=" * 60)
df[numerical_cols].describe()


## 5. Categorical Variables Analysis


In [ ]:
# Analyze categorical variables
categorical_cols = df.select_dtypes(include=['object']).columns

for col in categorical_cols:
    print(f"\n{col}:")
    print(f"  Unique values: {df[col].nunique()}")
    print(f"  Top 5 values:")
    print(df[col].value_counts().head().to_string())
    print("-" * 40)


## 6. Temporal Analysis (Year-wise Trends)


In [ ]:
# Analyze trends over years
if 'Year' in df.columns:
    yearly_stats = df.groupby('Year').agg({
        'Production': ['sum', 'mean', 'std'],
        'Quantity': ['mean', 'std'],
        'Cost': ['mean', 'std']
    }).round(2)
    
    print("Year-wise Statistics:")
    print(yearly_stats)
    
    # Visualize production trends
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Total production over years
    yearly_production = df.groupby('Year')['Production'].sum()
    axes[0, 0].plot(yearly_production.index, yearly_production.values, marker='o', linewidth=2)
    axes[0, 0].set_title('Total Production Over Years', fontsize=14, fontweight='bold')
    axes[0, 0].set_xlabel('Year')
    axes[0, 0].set_ylabel('Total Production (Tons)')
    axes[0, 0].grid(True, alpha=0.3)
    
    # Average yield over years
    yearly_yield = df.groupby('Year')['Quantity'].mean()
    axes[0, 1].plot(yearly_yield.index, yearly_yield.values, marker='s', color='green', linewidth=2)
    axes[0, 1].set_title('Average Yield Over Years', fontsize=14, fontweight='bold')
    axes[0, 1].set_xlabel('Year')
    axes[0, 1].set_ylabel('Average Yield (Quintals/Hectare)')
    axes[0, 1].grid(True, alpha=0.3)
    
    # Average cost over years
    yearly_cost = df.groupby('Year')['Cost'].mean()
    axes[1, 0].plot(yearly_cost.index, yearly_cost.values, marker='^', color='red', linewidth=2)
    axes[1, 0].set_title('Average Cost Over Years', fontsize=14, fontweight='bold')
    axes[1, 0].set_xlabel('Year')
    axes[1, 0].set_ylabel('Average Cost')
    axes[1, 0].grid(True, alpha=0.3)
    
    # Production distribution by year
    df.boxplot(column='Production', by='Year', ax=axes[1, 1])
    axes[1, 1].set_title('Production Distribution by Year', fontsize=14, fontweight='bold')
    axes[1, 1].set_xlabel('Year')
    axes[1, 1].set_ylabel('Production (Tons)')
    
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ 'Year' column not found in dataset")


## 7. State-wise Analysis


In [ ]:
# Analyze state-wise patterns
if 'State' in df.columns:
    state_stats = df.groupby('State').agg({
        'Production': 'sum',
        'Quantity': 'mean',
        'Cost': 'mean'
    }).sort_values('Production', ascending=False)
    
    print("Top 10 States by Total Production:")
    print(state_stats.head(10))
    
    # Visualize top states
    top_states = state_stats.head(10)
    
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    
    # Production by state
    axes[0].barh(top_states.index, top_states['Production'], color='skyblue')
    axes[0].set_title('Top 10 States by Total Production', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Total Production (Tons)')
    axes[0].invert_yaxis()
    
    # Average yield by state
    top_yield_states = state_stats.sort_values('Quantity', ascending=False).head(10)
    axes[1].barh(top_yield_states.index, top_yield_states['Quantity'], color='lightgreen')
    axes[1].set_title('Top 10 States by Average Yield', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Average Yield (Quintals/Hectare)')
    axes[1].invert_yaxis()
    
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ 'State' column not found in dataset")


## 8. Crop-wise Analysis


In [ ]:
# Analyze crop-wise patterns
if 'Crop' in df.columns:
    crop_stats = df.groupby('Crop').agg({
        'Production': 'sum',
        'Quantity': 'mean',
        'Cost': 'mean'
    }).sort_values('Production', ascending=False)
    
    print("Crop-wise Statistics:")
    print(crop_stats)
    
    # Visualize crop patterns
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    
    # Production by crop
    top_crops = crop_stats.head(10)
    axes[0].barh(top_crops.index, top_crops['Production'], color='coral')
    axes[0].set_title('Top 10 Crops by Total Production', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Total Production (Tons)')
    axes[0].invert_yaxis()
    
    # Average yield by crop
    top_yield_crops = crop_stats.sort_values('Quantity', ascending=False).head(10)
    axes[1].barh(top_yield_crops.index, top_yield_crops['Quantity'], color='gold')
    axes[1].set_title('Top 10 Crops by Average Yield', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Average Yield (Quintals/Hectare)')
    axes[1].invert_yaxis()
    
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ 'Crop' column not found in dataset")


## 9. Seasonal Analysis


In [ ]:
# Analyze seasonal patterns
if 'Season' in df.columns:
    season_stats = df.groupby('Season').agg({
        'Production': 'sum',
        'Quantity': 'mean',
        'Cost': 'mean'
    })
    
    print("Seasonal Statistics:")
    print(season_stats)
    
    # Visualize seasonal patterns
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    # Production by season
    axes[0].bar(season_stats.index, season_stats['Production'], color=['#FF6B6B', '#4ECDC4', '#45B7D1'])
    axes[0].set_title('Total Production by Season', fontsize=14, fontweight='bold')
    axes[0].set_ylabel('Total Production (Tons)')
    axes[0].tick_params(axis='x', rotation=45)
    
    # Average yield by season
    axes[1].bar(season_stats.index, season_stats['Quantity'], color=['#FF6B6B', '#4ECDC4', '#45B7D1'])
    axes[1].set_title('Average Yield by Season', fontsize=14, fontweight='bold')
    axes[1].set_ylabel('Average Yield (Quintals/Hectare)')
    axes[1].tick_params(axis='x', rotation=45)
    
    # Average cost by season
    axes[2].bar(season_stats.index, season_stats['Cost'], color=['#FF6B6B', '#4ECDC4', '#45B7D1'])
    axes[2].set_title('Average Cost by Season', fontsize=14, fontweight='bold')
    axes[2].set_ylabel('Average Cost')
    axes[2].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ 'Season' column not found in dataset")


## 10. Correlation Analysis


In [ ]:
# Correlation matrix for numerical features
numerical_df = df[numerical_cols]

if len(numerical_cols) > 1:
    correlation_matrix = numerical_df.corr()
    
    # Visualize correlation matrix
    plt.figure(figsize=(12, 8))
    sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
                center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
    plt.title('Correlation Matrix of Numerical Features', fontsize=16, fontweight='bold', pad=20)
    plt.tight_layout()
    plt.show()
    
    # Find strong correlations
    print("\nStrong Correlations (|r| > 0.5):")
    print("=" * 60)
    for i in range(len(correlation_matrix.columns)):
        for j in range(i+1, len(correlation_matrix.columns)):
            corr_value = correlation_matrix.iloc[i, j]
            if abs(corr_value) > 0.5:
                print(f"{correlation_matrix.columns[i]} <-> {correlation_matrix.columns[j]}: {corr_value:.3f}")
else:
    print("⚠️ Not enough numerical columns for correlation analysis")


## 11. Outlier Detection


In [ ]:
# Detect outliers using IQR method
def detect_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

# Check for outliers in key numerical columns
key_columns = ['Production', 'Quantity', 'Cost'] if all(col in df.columns for col in ['Production', 'Quantity', 'Cost']) else numerical_cols[:3]

outlier_summary = []
for col in key_columns:
    if col in df.columns:
        outliers, lower, upper = detect_outliers_iqr(df, col)
        outlier_summary.append({
            'Column': col,
            'Outliers Count': len(outliers),
            'Outlier Percentage': (len(outliers) / len(df)) * 100,
            'Lower Bound': lower,
            'Upper Bound': upper
        })

if outlier_summary:
    outlier_df = pd.DataFrame(outlier_summary)
    print("Outlier Detection Summary:")
    print(outlier_df.to_string(index=False))
    
    # Visualize outliers
    fig, axes = plt.subplots(1, len(key_columns), figsize=(15, 5))
    if len(key_columns) == 1:
        axes = [axes]
    
    for idx, col in enumerate(key_columns):
        if col in df.columns:
            df.boxplot(column=col, ax=axes[idx])
            axes[idx].set_title(f'Outliers in {col}', fontsize=12, fontweight='bold')
            axes[idx].set_ylabel(col)
    
    plt.tight_layout()
    plt.show()


## 12. Key Insights and Summary

### Findings:
1. **Dataset Size:** [X] records covering [X] years
2. **Geographic Coverage:** [X] states across India
3. **Crop Diversity:** [X] different crop types
4. **Temporal Range:** 2001-2014 (14 years of data)
5. **Data Quality:** [Missing values, outliers status]

### Patterns Identified:
- [Add key patterns from your analysis]
- [Add temporal trends]
- [Add geographic variations]
- [Add crop-specific insights]

### Recommendations for Preprocessing:
1. Handle missing values using [method]
2. Remove outliers using IQR method for [columns]
3. Encode categorical variables: Crop, State, Season
4. Create derived features: Year_Squared, Cost_per_Unit
5. Normalize/scale features if needed

### Next Steps:
- Proceed to data preprocessing (Week 2)
- Implement feature engineering
- Prepare data for model training
